In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import lfilter
from scipy.signal.windows import hamming

from numpy.linalg import lstsq

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

zip_path="/content/drive/MyDrive/L2_DATASET.zip"

import zipfile,os,glob
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display

extract_path="/content/voices_dataset"
os.makedirs(extract_path,exist_ok=True)
with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped to:",extract_path)
print("Files:",os.listdir(extract_path))

print("Extracted folder content:")
for r,ds,fs in os.walk(extract_path):
    print(r,"→",fs)

# --- dataset paths ---
extract_path="/content/voices_dataset"
audio_dir=os.path.join(extract_path,"L2_DATASET")
all_audio=sorted(glob.glob(os.path.join(audio_dir,"*.wav")))
print(f"Found {len(all_audio)} WAV files")
assert all_audio, "No WAV files found. Check /content/voices_dataset/L2_DATASET"

# Helper: plot/save narrowband & wideband spectrograms ----
def plot_spectrograms(file_path, show=False, save_dir=None):
    y, sr=librosa.load(file_path, sr=None, mono=True)

    # --- Narrowband: large FFT window ---
    D_narrow=librosa.stft(y, n_fft=2048, hop_length=512)
    S_narrow=librosa.amplitude_to_db(np.abs(D_narrow), ref=np.max)
    plt.figure(figsize=(10,4))
    librosa.display.specshow(S_narrow, sr=sr, hop_length=512, x_axis='time', y_axis='hz')
    plt.colorbar(label="dB")
    plt.title(f"Narrowband Spectrogram — {os.path.basename(file_path)}")
    plt.ylim(0,3000)
    plt.tight_layout()
    if save_dir:
        save_path=os.path.join(save_dir, os.path.splitext(os.path.basename(file_path))[0]+"_narrow.png")
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    elif show:
        plt.show()
    else:
        plt.close()

    # Wideband: small FFT window
    D_wide=librosa.stft(y, n_fft=512, hop_length=128)
    S_wide=librosa.amplitude_to_db(np.abs(D_wide), ref=np.max)
    plt.figure(figsize=(10,4))
    librosa.display.specshow(S_wide, sr=sr, hop_length=128, x_axis='time', y_axis='hz')
    plt.colorbar(label="dB")
    plt.title(f"Wideband Spectrogram — {os.path.basename(file_path)}")
    plt.ylim(0,4000)
    plt.tight_layout()
    if save_dir:
        save_path=os.path.join(save_dir, os.path.splitext(os.path.basename(file_path))[0]+"_wide.png")
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    elif show:
        plt.show()
    else:
        plt.close()

#  1) Show example for first file ----
first_file=all_audio[0]
y, sr=librosa.load(first_file, sr=None, mono=True)
display(Audio(y, rate=sr))
plot_spectrograms(first_file, show=True)

#  2) Loop over all files and save plots ----
out_dir=os.path.join(extract_path,"bandwidth_spectrograms")
os.makedirs(out_dir, exist_ok=True)

for fp in all_audio:
    plot_spectrograms(fp, save_dir=out_dir)

print(f" Saved narrowband & wideband spectrograms for {len(all_audio)} files to: {out_dir}")
